# Reading / Writing files from S3 in CML

##  Option 1 - USE  SPARK
### **Applicable to:**  All datasets and large ones in particular <br> 
IDBroker integration allows direct access to the S3 buckets associated with the environement

### Use Case 1. Reading data from an S3 bucket associated with the Environement 

### Start the spark session
Custom session configuration can be defined either in the session parameters as below OR
inside a `spark-defaults.conf` file stored at the root of the project (in which case the configs become project wide)

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession\
  .builder\
  .appName('Airline')\
  .config("spark.executor.memory","2g")\
  .config("spark.executor.cores","2")\
  .config("spark.executor.instances","3")\
  .config("spark.yarn.access.hadoopFileSystems","s3a://prod-cdptrialuser19-trycdp-com/cdp-lake/")\
  .getOrCreate()

In [2]:
## Adding a link to the Spark UI for demo purposes
## Also available in the session tab
from IPython.core.display import HTML
import os
HTML('<a href="http://spark-{}.{}" target="_blank" >Spark UI</a>'.\
     format(os.getenv("CDSW_ENGINE_ID"),os.getenv("CDSW_DOMAIN")))

### Read Data - CSV file stored on HDFS 

In [3]:
path='airlines/airports/airports.csv' #HDFS location

airports_df = spark.read.csv(
    path=path,
    header=True,
    sep=',',
    inferSchema=True,
    nullValue=None
)
airports_df.printSchema()

root
 |-- iata: string (nullable = true)
 |-- airport: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)



**Note** : in the above example, I'm infering the schema from the file. <br>
It's actually good practice to set the schema to prevent erroneous type casting

In [4]:
from pyspark.sql.types import *

path='airlines/airports/airports.csv' #HDFS location
schema = StructType([StructField("iata", StringType(), True),
                     StructField("airport", StringType(), True),
                     StructField("city", StringType(), True),
                     StructField("state", StringType(), True),
                     StructField("country", StringType(), True),
                     StructField("lat",  DoubleType(), True),
                     StructField("long",  DoubleType(), True)
                    ])

airports_df = spark.read.csv(
    path=path,
    schema=schema,
    header=True,
    sep=',',
    nullValue=None
).cache()
airports_df.show(5)

+----+--------------------+----------------+-----+-------+-----------+------------+
|iata|             airport|            city|state|country|        lat|        long|
+----+--------------------+----------------+-----+-------+-----------+------------+
| 00M|            Thigpen |     Bay Springs|   MS|    USA|31.95376472|-89.23450472|
| 00R|Livingston Municipal|      Livingston|   TX|    USA|30.68586111|-95.01792778|
| 00V|         Meadow Lake|Colorado Springs|   CO|    USA|38.94574889|-104.5698933|
| 01G|        Perry-Warsaw|           Perry|   NY|    USA|42.74134667|-78.05208056|
| 01J|    Hilliard Airpark|        Hilliard|   FL|    USA| 30.6880125|-81.90594389|
+----+--------------------+----------------+-----+-------+-----------+------------+
only showing top 5 rows



### (optional) Transform data to Pandas Dataframe
#### Once converted **ALL DATA will be brought locally** and distributed processing ends 
* **Applicable to : SMALL to MEDIUM size datasets** - ie : datasets that can be easily managed/processed locally
* When working with **LARGE datasets** :  **data should be sampled** before bringing it locally

> **Good Practice**:  Spark context should be stopped `spark.stop()` to release cluster ressources once data is copied

In [5]:
#without sampling
import pandas 
airport_pandas_df = airports_df.toPandas()
airport_pandas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3376 entries, 0 to 3375
Data columns (total 7 columns):
iata       3376 non-null object
airport    3376 non-null object
city       3376 non-null object
state      3376 non-null object
country    3376 non-null object
lat        3376 non-null float64
long       3376 non-null float64
dtypes: float64(2), object(5)
memory usage: 184.7+ KB


In [6]:
#with sampling
sample_pandas_df = airports_df.sample(1/3,seed=30).toPandas()
sample_pandas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1111 entries, 0 to 1110
Data columns (total 7 columns):
iata       1111 non-null object
airport    1111 non-null object
city       1111 non-null object
state      1111 non-null object
country    1111 non-null object
lat        1111 non-null float64
long       1111 non-null float64
dtypes: float64(2), object(5)
memory usage: 60.8+ KB


### 2. Write data from PANDAS to HDFS - Using Spark

#### Read the data using Pandas

In [7]:
## read from pandas
import pandas as pd
airlines_pd_df = pd.read_csv("/home/cdsw/airlines/airports/airports.csv",sep=',', delimiter=None, header='infer')
airlines_pd_df.sort_values(by=['state','airport'],inplace=True) # ordering to keep same visulisation order 
airlines_pd_df.head()

,iata,airport,city,state,country,lat,long
776,ADK,Adak,Adak,AK,USA,51.877964,-176.646031
818,AKK,Akhiok,Akhiok,AK,USA,56.938691,-154.182556
3363,Z13,Akiachak,Akiachak,AK,USA,60.904532,-161.420910
817,AKI,Akiak,Akiak,AK,USA,60.904812,-161.227019
1994,KQA,Akutan SPB,Akutan,AK,USA,54.132467,-165.785311


### Transform Pandas DataFrame to Spark DataFrame
With spark 2.3 and up, integration with Pandas has been reinforced notably with the use of Arrow for faster data transfers [https://issues.apache.org/jira/browse/SPARK-20791]

In [8]:
# (optional) Enable Arrow-based optimised columnar data transfers ; Note : still marked as experimental
# spark.conf.set("spark.sql.execution.arrow.enabled", "true") # Note : Compatible only with pyarrow 0.8.0 
# https://spark.apache.org/docs/latest/sql-pyspark-pandas-with-arrow.html#ensure-pyarrow-installed

#(optional) good practice to define schema to prevent any type casting errors
from pyspark.sql.types import *

schema = StructType([StructField("iata", StringType(), True),
                     StructField("airport", StringType(), True),
                     StructField("city", StringType(), True),
                     StructField("state", StringType(), True),
                     StructField("country", StringType(), True),
                     StructField("lat",  DoubleType(), True),
                     StructField("long",  DoubleType(), True)
                    ])

spark_df=spark.createDataFrame(airlines_pd_df,schema=schema)
spark_df.orderBy(['state','airport']).show(5) # ordering to keep same visulisation order 

+----+----------+--------+-----+-------+-----------+-------------------+
|iata|   airport|    city|state|country|        lat|               long|
+----+----------+--------+-----+-------+-----------+-------------------+
| ADK|      Adak|    Adak|   AK|    USA|51.87796389|       -176.6460306|
| AKK|    Akhiok|  Akhiok|   AK|    USA|56.93869083|       -154.1825556|
| Z13|  Akiachak|Akiachak|   AK|    USA|60.90453167|-161.42091000000002|
| AKI|     Akiak|   Akiak|   AK|    USA|60.90481194|       -161.2270189|
| KQA|Akutan SPB|  Akutan|   AK|    USA|54.13246694|       -165.7853111|
+----+----------+--------+-----+-------+-----------+-------------------+
only showing top 5 rows



### Write to HDFS - using Spark

In [10]:
## It's good practice to restructure data before writing to HDFS : Spark write a file by partition. 
## this can lead to lots of small files which is counterproductive both for read and write. 
## Re-organize data using the "coalesce" function to define the number of files to be saved
location = 's3a://prod-cdptrialuser19-trycdp-com/cdp-lake/data/cml_rw_basics/tmp/airlines'

spark_df.coalesce(2).write.parquet(location, mode='overwrite')

test_file_df = spark.read.parquet(location)
test_file_df.printSchema()

root
 |-- iata: string (nullable = true)
 |-- airport: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)



### Write Data to Hive - using Spark
Spark to hive integration makes it very easy to interact with the cluster. 

In [11]:
# Note : Ordering on write can help optimise reads later on. 
spark_df.orderBy(['state','airport']).coalesce(2)\
    .write.format('parquet').mode("overwrite")\
    .saveAsTable('flights.airports_new')

### 3. Read Data from Hive 
All hive configurations are already injected into spark.  Therefore Hive can be called directly using a spark sql context.

In [12]:
sql_statement = '''show tables in flights'''
spark.sql(sql_statement).show()

+--------+------------+-----------+
|database|   tableName|isTemporary|
+--------+------------+-----------+
| flights|    airports|      false|
| flights|airports_new|      false|
| flights|    carriers|      false|
| flights| flights_raw|      false|
+--------+------------+-----------+



In [13]:
#read table
sql_statement = '''select * from flights.airports where state = "AK" '''
airports_df = spark.sql(sql_statement)
airports_df.show(10)

+----+----------+--------------+-----+-------+-----------+------------+
|iata|   airport|          city|state|country|        lat|        long|
+----+----------+--------------+-----+-------+-----------+------------+
| ADK|      Adak|          Adak|   AK|    USA|51.87796389|-176.6460306|
| AKK|    Akhiok|        Akhiok|   AK|    USA|56.93869083|-154.1825556|
| Z13|  Akiachak|      Akiachak|   AK|    USA|60.90453167|  -161.42091|
| AKI|     Akiak|         Akiak|   AK|    USA|60.90481194|-161.2270189|
| KQA|Akutan SPB|        Akutan|   AK|    USA|54.13246694|-165.7853111|
| AUK|  Alakanuk|      Alakanuk|   AK|    USA|62.68004417|-164.6599253|
| 5A8| Aleknagik|     Aleknagik|   AK|    USA|59.28256167|-158.6176725|
| 6A8| Allakaket|     Allakaket|   AK|    USA|66.55194444|-152.6222222|
| BIG| Allen AAF|Delta Junction|   AK|    USA|63.99454722|-145.7216417|
| AFM|    Ambler|        Ambler|   AK|    USA|67.10610472|-157.8536203|
+----+----------+--------------+-----+-------+-----------+------

In [14]:
#(OPTIONAL) convert to pandas 
airlines_pd_df = airports_df.toPandas()
airlines_pd_df.head()

,iata,airport,city,state,country,lat,long
0,ADK,Adak,Adak,AK,USA,51.877964,-176.646031
1,AKK,Akhiok,Akhiok,AK,USA,56.938691,-154.182556
2,Z13,Akiachak,Akiachak,AK,USA,60.904532,-161.420910
3,AKI,Akiak,Akiak,AK,USA,60.904812,-161.227019
4,KQA,Akutan SPB,Akutan,AK,USA,54.132467,-165.785311


In [15]:
spark.stop() ## Release spark ressources

#### ***NOTE:*** Pandas Dataframe is still available

In [16]:
airport_pandas_df.head()

,iata,airport,city,state,country,lat,long
0,00M,Thigpen,Bay Springs,MS,USA,31.953765,-89.234505
1,00R,Livingston Municipal,Livingston,TX,USA,30.685861,-95.017928
2,00V,Meadow Lake,Colorado Springs,CO,USA,38.945749,-104.569893
3,01G,Perry-Warsaw,Perry,NY,USA,42.741347,-78.052081
4,01J,Hilliard Airpark,Hilliard,FL,USA,30.688012,-81.905944


## OPTION 2 - Read_Write Directly from Pandas ( using AWS secret and key ) 


### Pandas can read directly from S3
Dependencies : s3fs library
> Note : AWS CLI must be configured ahead of time with AWS key and secret

In [24]:
s3_loc = 's3://ml-field/demo/wine/WineNewGBTDataSet.csv'
airlines_pd_df = pd.read_csv(s3_loc,sep=';', delimiter=None, header='infer')
airlines_pd_df.head()

,7.4,0.7,0,1.9,0.076,11,34,0.9978,3.51,0.56,9.4,Poor
0,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,Poor
1,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,Poor
2,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,Excellent
3,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,Poor
4,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4,Poor
